Przygotowanie danych tak, jak ustalilismy w poprzedniej częsci (usuwamy outliery i niektóre kolumny z high correlation).

In [11]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [2]:
df_train = pd.read_csv('../../data/for_modelling/credit_score_train.csv')
df_val = pd.read_csv('../../data/for_modelling/credit_score_valid.csv')
df=pd.concat([df_train, df_val])
target='DEFAULT'
#df=df_train.copy()
df

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.0000,0.0000,103,74,0.7184,0.0370,...,2.5003,0.0000,No,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.2500,1.5882,35861,29157,0.8131,0.1141,...,0.5882,0.3704,High,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.2000,5.3750,3716,2533,0.6816,0.0230,...,0.3125,0.0581,No,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.3000,0.2131,637,187,0.2936,0.0398,...,0.1639,0.7692,No,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.4000,1.8356,5276,2325,0.4407,0.0273,...,0.1370,0.0746,No,1,1,0,1,1,552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,386976,1250231,654883,3.2308,1.6923,0.5238,36665,30569,0.8337,0.0947,...,0.2381,0.4545,No,1,0,0,1,0,562,0
134,16772,8386,0,0.5000,0.0000,0.0000,238,186,0.7815,0.0142,...,2.5000,0.9786,No,0,0,0,1,0,619,0
135,137509,206264,206264,1.5000,1.5000,1.0000,15661,10865,0.6938,0.1139,...,0.6667,0.6667,No,1,0,0,1,0,639,1
136,259146,1922000,1662854,7.4167,6.4167,0.8652,12180,8527,0.7001,0.0470,...,0.1124,0.1299,High,1,1,0,1,1,587,1


In [3]:
# columns to remove from EDA
columns_to_remove = ["T_CLOTHING_12","T_ENTERTAINMENT_12", "T_GROCERIES_12", "T_GROCERIES_6", "T_HEALTH_12", "T_TAX_12", "T_TAX_6", "T_TRAVEL_12", "T_TRAVEL_6","T_UTILITIES_12", "T_UTILITIES_6", "T_EXPENDITURE_12", "T_EXPENDITURE_6"]
columns_to_remove

['T_CLOTHING_12',
 'T_ENTERTAINMENT_12',
 'T_GROCERIES_12',
 'T_GROCERIES_6',
 'T_HEALTH_12',
 'T_TAX_12',
 'T_TAX_6',
 'T_TRAVEL_12',
 'T_TRAVEL_6',
 'T_UTILITIES_12',
 'T_UTILITIES_6',
 'T_EXPENDITURE_12',
 'T_EXPENDITURE_6']

In [4]:

# map CAT_GAMBLING (No-0, Low-1, High-2)
df_train['CAT_GAMBLING'] = df_train['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
df_val['CAT_GAMBLING'] = df_val['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
# remove columns with high correlation
df_train.drop(columns=columns_to_remove, inplace=True)
df_val.drop(columns=columns_to_remove, inplace=True)
# remove outliers
from pyod.models.knn import KNN
clf = KNN(contamination=0.04)
rest_cols = [col for col in df_train.columns if col not in columns_to_remove]
# remove DEFAULT column
rest_cols.remove('DEFAULT')
clf.fit(df_train[rest_cols])
df_train['outliers'] = clf.labels_
# sum of outliers
df_train['outliers'].value_counts()

0    470
1     20
Name: outliers, dtype: int64

In [5]:
df_train = df_train[df_train['outliers'] == 0]
df_train.drop(columns='outliers', inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,R_CLOTHING_SAVINGS,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.0000,0.0000,74,0.7184,0.0370,0.0555,...,2.5003,0.0000,0,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.2500,1.5882,29157,0.8131,0.1141,0.0805,...,0.5882,0.3704,2,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.2000,5.3750,2533,0.6816,0.0230,0.0072,...,0.3125,0.0581,0,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.3000,0.2131,187,0.2936,0.0398,0.0065,...,0.1639,0.7692,0,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.4000,1.8356,2325,0.4407,0.0273,0.0037,...,0.1370,0.0746,0,1,1,0,1,1,552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,20007,22508,457665,1.1250,22.8752,20.3334,861,0.5934,0.0725,0.0645,...,1.1111,0.0546,0,1,0,0,1,0,423,0
466,20846,268679,23162,12.8888,1.1111,0.0862,533,0.4094,0.0625,0.0048,...,0.0862,1.0000,1,1,0,0,1,0,584,0
467,89235,1106514,160623,12.4000,1.8000,0.1452,835,0.3460,0.0270,0.0022,...,0.0806,0.5556,0,1,0,0,1,0,631,0
468,197073,1021195,1863233,5.1818,9.4545,1.8246,3948,0.4691,0.0427,0.0082,...,0.1754,0.0962,2,1,0,1,1,1,572,0


Dalej przygotowaujemy dane: transformacja Boxa Coxa i standaryzacja.

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import boxcox

X=df_train.drop(target, axis=1)
y=df_train[target]
X=X[rest_cols]
X_val=df_val.drop(target, axis=1)
y_val=df_val[target]
X_val=X_val[rest_cols]

# Box Cox and standart scalling
cox_box_model = PowerTransformer(method='yeo-johnson')
X = cox_box_model.fit_transform(X)
X_val = cox_box_model.transform(X_val)
X = scaler.fit_transform(X)
X_val = scaler.transform(X_val)


Dane gotowe. Teraz sprawdzamy kilka modeli.

# Komitety

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier

model1 = DecisionTreeClassifier(random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
model5 = RandomForestClassifier(random_state=1)
model6 = GradientBoostingClassifier(random_state=1)
model8 = GaussianNB()


estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model3),  ('RandomForest', model5),
            ('GradientBoosting', model6),
            ('NaiveBayes', model8)]

## hard voting

In [14]:
from sklearn.ensemble import VotingClassifier
model_hard = VotingClassifier(estimators=estimators, voting='hard')
model_hard.fit(X,y)

y_hat = model_hard.predict(X_val)
print('f1 score: ', f1_score(y_val, y_hat))

print('model.score: ', model_hard.score(X_val,y_val))
report = classification_report(y_val, y_hat)
print(report)

f1 score:  0.3448275862068966
model.score:  0.7246376811594203
              precision    recall  f1-score   support

           0       0.76      0.91      0.83        99
           1       0.53      0.26      0.34        39

    accuracy                           0.72       138
   macro avg       0.64      0.58      0.59       138
weighted avg       0.69      0.72      0.69       138


## soft voting

In [15]:
model_soft = VotingClassifier(estimators=estimators, voting='soft')
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
f1_score(y_val, y_hat)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.82      0.78        99
           1       0.40      0.31      0.35        39

    accuracy                           0.67       138
   macro avg       0.57      0.56      0.57       138
weighted avg       0.65      0.67      0.66       138


## soft voting z wagami (tak bardziej dla sprawdzenia jak to dziala)

In [16]:
from sklearn.metrics import classification_report
model_soft = VotingClassifier(estimators=estimators, voting='soft', weights=[0.1, 0.1, 0.3, 0.3, 0.1,0.2])
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.77      0.79      0.78        99
           1       0.43      0.41      0.42        39

    accuracy                           0.68       138
   macro avg       0.60      0.60      0.60       138
weighted avg       0.68      0.68      0.68       138


## stacking

In [18]:
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X, y).score(X_val, y_val)
y_hat = clf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)


              precision    recall  f1-score   support

           0       0.75      0.95      0.84        99
           1       0.58      0.18      0.27        39

    accuracy                           0.73       138
   macro avg       0.66      0.56      0.56       138
weighted avg       0.70      0.73      0.68       138


## bagging

In [19]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(estimator=model5
                        ,n_estimators=10, random_state=0)
clf.fit(X, y)
y_hat = clf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.91      0.81        99
           1       0.44      0.18      0.25        39

    accuracy                           0.70       138
   macro avg       0.59      0.54      0.53       138
weighted avg       0.65      0.70      0.66       138


sprawdzilam wszystkie modele, model5 najlepszy

## random forest classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=3, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X, y)
y_hat = model_rf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.99      0.83        99
           1       0.50      0.03      0.05        39

    accuracy                           0.72       138
   macro avg       0.61      0.51      0.44       138
weighted avg       0.66      0.72      0.61       138


In [22]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.75      0.81      0.78        99
           1       0.41      0.33      0.37        39

    accuracy                           0.67       138
   macro avg       0.58      0.57      0.57       138
weighted avg       0.66      0.67      0.66       138


In [23]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=1,
                                   learning_rate=0.01)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.96      0.84        99
           1       0.60      0.15      0.24        39

    accuracy                           0.73       138
   macro avg       0.67      0.56      0.54       138
weighted avg       0.70      0.73      0.67       138


In [24]:
from xgboost import XGBClassifier
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.96      0.83        99
           1       0.56      0.13      0.21        39

    accuracy                           0.72       138
   macro avg       0.65      0.54      0.52       138
weighted avg       0.69      0.72      0.66       138


In [25]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
level0 = [
    ('lr', LogisticRegression()),
    ('svc', SVC()),
    ('dt', DecisionTreeClassifier())
]
# Meta-model, który podsumowuje przewidywania bazowych modeli
level1 = LogisticRegression()

# Defining the stacking model
model_stacking = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [27]:
# fitting stacking and evaluating
model_stacking.fit(X, y)
y_hat = model.predict(X_val)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.96      0.83        99
           1       0.56      0.13      0.21        39

    accuracy                           0.72       138
   macro avg       0.65      0.54      0.52       138
weighted avg       0.69      0.72      0.66       138


## strojenie parametrów

decision tree

In [28]:
tree_model= DecisionTreeClassifier()

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
tree_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [34]:
max_depth=[3, 5, 6, 7, 8]
criterion=["gini","entrophy"]
ccp_alpha=[0,0.01, 0.02, 0.05, 0.1, 0.2]
splitter=["best", "random"]
min_samples_split=[2, 5, 10]
min_samples_leaf=[1, 2, 4]
max_features=["auto", "sqrt", "log2"]

param_grid = {
    'criterion': criterion,
    'splitter': splitter,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'ccp_alpha': ccp_alpha,
    'max_features': max_features,
}


In [36]:
tree_model= DecisionTreeClassifier()
grid = GridSearchCV(estimator=tree_model, param_grid=param_grid, cv = 3, n_jobs=-1, scoring='f1')

grid_result = grid.fit(X, y) #tutaj lepiej zastosować tylko trainset

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.402668 using {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'splitter': 'best'}


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6480 fits failed out of a total of 9720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
884 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Dell\anaconda3\Lib\site-packages\

In [38]:
std_test_scores = grid_result.cv_results_['std_test_score']
mean_test_scores = grid_result.cv_results_['mean_test_score']
parameters = grid_result.cv_results_['params']

for mean_test, std_test, params in zip(mean_test_scores, std_test_scores, parameters):
    print('Parameters : {}'.format(params))
    print('Mean test score: {:.8f}'.format(mean_test))
    print('Standard Deviation (test) : {:.8f}'.format(std_test))
    print()

print('Best parameters : {}'.format(grid_result.best_params_))
print('Best accuracy score : {:.8f}'.format(grid_result.best_score_))


Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'random'}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter

In [39]:
best_model=grid_result.best_estimator_
best_model.score(X_val, y_val)

0.6376811594202898

In [40]:
best_model.fit(X, y)
y_hat = best_model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.86      0.79        99
           1       0.36      0.21      0.26        39

    accuracy                           0.67       138
   macro avg       0.55      0.53      0.53       138
weighted avg       0.63      0.67      0.64       138


dla Gradient Boosting

In [42]:
max_depth = [3, 5, 6, 7, 8]
loss = ["deviance", "exponential"]
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [10, 50, 100, 200]
subsample = [0.5, 0.75, 1.0]
max_features = ["auto", "sqrt", "log2"]

param_grid = {
    'loss': loss,
    'learning_rate': learning_rate,
    'n_estimators': n_estimators,
    'subsample': subsample,
    'max_depth': max_depth,
    'max_features': max_features,
    'ccp_alpha': ccp_alpha
}

In [43]:
gb_model = GradientBoostingClassifier()

grid = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv = 3, n_jobs=-1, scoring='f1')

grid_result = grid.fit(X, y) 

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
12960 fits failed out of a total of 19440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Dell\anaconda3\Lib\site-packag

Best: 0.393738 using {'ccp_alpha': 0, 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 100, 'subsample': 0.5}


In [49]:
y_hat=grid_result.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.82      0.77        99
           1       0.33      0.23      0.27        39

    accuracy                           0.65       138
   macro avg       0.53      0.52      0.52       138
weighted avg       0.62      0.65      0.63       138


dla random forest 

In [50]:
max_depth = [3, 5, 6, 7, 8]
criterion = ["gini", "entropy"]
ccp_alpha = [0, 0.01, 0.02, 0.05, 0.1, 0.2]
max_features = ["auto", "sqrt", "log2"]
n_estimators = [10, 50, 100, 200]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap,
    'criterion': criterion,
    'ccp_alpha': ccp_alpha
}

In [ ]:
rf_model = RandomForestClassifier()

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1')

grid_result = grid.fit(X, y)

print("Best F1-score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
y_hat=grid_result.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

dla naive bayes

In [ ]:
from sklearn.model_selection import StratifiedKFold


# Ustal parametry i model
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

nb_model = GaussianNB()

# Utwórz GridSearchCV z użyciem StratifiedKFold i własnej funkcji scoring
grid = GridSearchCV(estimator=nb_model, param_grid=param_grid, cv=StratifiedKFold(3), n_jobs=-1)

# Dopasuj GridSearchCV do danych treningowych
grid_result = grid.fit(X_train, y_train)

# Oblicz f1-score dla najlepszego modelu
y_pred = grid_result.best_estimator_.predict(X_test)
f1 = f1_score(y_test, y_pred)

# Wydrukuj najlepszy wynik i najlepsze parametry
print("Best F1-score: %f using %s" % (f1, grid_result.best_params_))

In [41]:
from sklearn.model_selection import RandomizedSearchCV

In [52]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.717023 using {'ccp_alpha': 3.180672781924487, 'max_depth': 1}


In [53]:
best_model=random_result.best_estimator_
best_model.score(X_val, y_val)

0.717391304347826

In [55]:
from scipy.stats import poisson,expon
param_grid = {'ccp_alpha': expon(0.08),
              'max_depth': poisson(5)}

In [56]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1, random_state=123)

random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.717023 using {'ccp_alpha': 1.2722721434811057, 'max_depth': 7}


In [46]:
! pip install scikit-optimize

  Obtaining dependency information for scikit-optimize from https://files.pythonhosted.org/packages/90/0e/15deb91b3db0003843e34e72fa865e1d92013781d986fdc65483c99a9f69/scikit_optimize-0.10.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyaml>=16.9 from https://files.pythonhosted.org/packages/70/d9/6fdc01cee01fe56fddfd46892d9846ebb53a5f8eb78056955c516eef42c5/pyaml-24.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   ----------- --------------------------- 30.7/107.7 kB 660.6 kB/s eta 0:00:01
   -------------------------------------- - 102.4/107.7 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- 107.7/107.7 kB 894.5 kB/s eta 0:00:00


In [57]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

opt = BayesSearchCV(
    SVC(),
    {
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=5, #generalnie powinno być więcej, ale wtedy długo trwa
    cv=3
)

opt.fit(X, y)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_val, y_val))

val. score: 0.7255566443464533
test score: 0.2826086956521739


In [63]:
!pip install -U lightautoml

  Obtaining dependency information for lightautoml from https://files.pythonhosted.org/packages/6d/82/81771067b1ecc2ac87aec5d14d862b4567eea95504ef576718f6b7755ac6/lightautoml-0.3.8.1-py3-none-any.whl.metadata
  Using cached lightautoml-0.3.8.1-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for autowoe>=1.2 from https://files.pythonhosted.org/packages/61/09/ccf2ad21f105d07d73bedf065597359a2445d053696a9e1e57dcee397a97/AutoWoE-1.3.2-py3-none-any.whl.metadata
  Using cached AutoWoE-1.3.2-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for catboost>=0.26.1 from https://files.pythonhosted.org/packages/e8/37/3afd3c02798734efcd7840bfa872d3efc06f5d5c92f9613fea3ff5b4311f/catboost-1.2.3-cp311-cp311-win_amd64.whl.metadata
  Using cached catboost-1.2.3-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/4d/c1/9190d856fb2c9ca7e5dabab530289bf66d03c0e379096678c0ee210abf75/ho

In [64]:
import os
import time
import re

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Imports from LightAutoML package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

In [65]:
def f1_metric(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

def acc_score(y_true, y_pred):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int))

def gini(y_true, y_pred):
    return 2*roc_auc_score(y_true, y_pred)-1

task = Task('binary', metric = f1_metric)

In [72]:
roles = {'target': 'DEFAULT'}

In [73]:
automl = TabularAutoML(task = task,
                       timeout = 600, # 600 seconds = 10 minutes
                       cpu_limit = 4, # 
                       general_params = {'use_algos': [['linear_l2',
                                                        'lgb', 'lgb_tuned']]})

In [82]:
tr_data=X
tr_data["DEFAULT"]=y
valid_data=X_val
valid_data["DEFAULT"]=y_val
oof_pred = automl.fit_predict(tr_data, roles = roles)

In [83]:
valid_pred = automl.predict(valid_data)

In [85]:
print('OOF acc: {}'.format(acc_score(tr_data['DEFAULT'].values,      oof_pred.data[:, 0])))
print('VAL acc: {}'.format(acc_score(valid_data['DEFAULT'].values, valid_pred.data[:, 0])))

OOF acc: 0.7489361702127659
VAL acc: 0.644927536231884


In [86]:
!pip install tpot

In [87]:
from tpot import TPOTClassifier

In [88]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X, y)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Best pipeline: GaussianNB(input_matrix)


TPOTClassifier(generations=5, population_size=50, random_state=42, verbosity=2)

In [89]:
print("Accuracy on testing data: ", tpot.score(X_val, y_val))

Accuracy on testing data:  0.717391304347826


In [90]:
!pip install boruta

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

array([[-1.67311484, -1.79479368, -2.13887769, ...,  0.06537205,
        -0.42532872, -0.43113655],
       [ 1.52579493,  0.52768342,  0.49449643, ...,  0.06537205,
        -0.42532872,  1.9855153 ],
       [ 0.6954251 ,  0.66687435,  1.55375508, ...,  0.06537205,
         2.35112266, -1.1557775 ],
       ...,
       [ 0.10246894,  1.48375   , -0.32246311, ...,  0.06537205,
        -0.42532872,  0.66123881],
       [ 0.92176584,  1.38769659,  1.20700545, ...,  0.06537205,
         2.35112266, -0.39908084],
       [ 0.49070335, -0.6318861 ,  0.46881365, ...,  0.06537205,
        -0.42532872,  0.25773124]])